In [15]:
from gurobipy import *
from math import sqrt
import pandas as pd
import numpy as np
size  =  20    #manja velicina za provjeru
sigma = np.genfromtxt("orl200-005-b.mat", skip_header = 1, dtype = 'int')
sigma = sigma[0:size,0:size]
mu = pd.read_csv("orl200-005-b.txt",sep = " ", index_col = False, skiprows = 1,names = ['mu_i', 'ignore'])
rho = np.genfromtxt("orl200-005-b.rho")
rho = rho.item()
mu = mu.iloc[0:size,0]
bounds = pd.read_csv("orl200-005-b.bds", sep = " ", index_col = False,names = ['l_i', 'u_i'])
bounds = bounds.transpose()
u_i = bounds.iloc[1,0:size]
m = Model('portfolio')
vars_ =  pd.Series(m.addVars(size))  #C_0, ..., C_199
y = pd.Series(m.addVars(size, vtype = GRB.BINARY))
vars_.start = np.zeros(size)
y.start = np.ones(size)
portfolio_risk = vars_.dot(sigma).dot(vars_)
m.setObjective(portfolio_risk, GRB.MINIMIZE)
m.addConstr(vars_.sum() <=  1, 'budget')
mu = np.asarray(mu)
m.addConstrs((0 <= vars_[i] for i in range(size)), 'nonnegative')
m.addConstrs((vars_[i] <= u_i[i]*y[i] for i in range(size)), 'u_i')
m.addConstr(y.sum() <= 5, 'cardinality')
portfolio_return= mu.dot(vars_)  #mu^T*x > =  ro
m.addConstr(rho <= mu.dot(vars_), 'portf_return')
m.addConstrs((y[i] <= 1 for i in range(size)), 'interval')
m.addConstrs((0 <= y[i] for i in range(size)), 'interval2')
m.update()


In [16]:
m.setParam('TimeLimit', 7200)
m.optimize()

Changed value of parameter TimeLimit to 7200.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 83 rows, 40 columns and 160 nonzeros
Model has 210 quadratic objective terms
Variable types: 20 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [2e-03, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [4e+00, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 5e+00]
Presolve removed 60 rows and 0 columns
Presolve time: 0.00s
Presolved: 23 rows, 40 columns, 100 nonzeros
Presolved model has 210 quadratic objective terms
Variable types: 20 continuous, 20 integer (20 binary)
Found heuristic solution: objective 112.4709817
Found heuristic solution: objective 108.6244049

Root relaxation: objective 3.222243e+01, 62 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   32.22243

In [12]:
for v in vars_:
    if v.x > 0:
        print('\t%s\t: %g' % (v.varname, v.x))

	C0	: 0.100453
	C3	: 0.115461
	C4	: 0.144964
	C6	: 0.132069
	C8	: 0.132671
